In [1]:
USE_SAME_FEW_SHOTS=True

In [2]:
!pip install -q openai #--progress-bar off

In [3]:
import pandas as pd

import datetime
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

import os
import json
import ast
import re

import pickle

import glob

# import openai
# from openai import OpenAI

from typing import Union
from IPython.display import display, HTML

from sklearn.metrics import classification_report

In [4]:
import os
from IPython.display import clear_output

os.environ["OPENAI_API_BASE"] = 'https://api.endpoints.anyscale.com/v1'
os.environ["OPENAI_API_KEY"] = input("INSERT YOUR ANYSCALE API KEY:")

clear_output()

In [5]:
output_datapath = "data/output/2023.11.22-1_Llama2_13B_API_7-few-shot/"

In [6]:
# # llm = openai.ChatCompletion.create(
# #     model = 'meta-llama/Llama-2-7b-chat-hf', 
# #     messages = [{"role": "system", "content": prompt}],
# #     max_tokens = 150,
# #     response_format = {"type": "json_object"},
# #     stream = True
# # )

# client = OpenAI()

# chat_completion = client.chat.completions.create(
#     messages=[
#         {
#             "role": "user",
#             "content": "Say this is a test",
#         }
#     ],
#     model='meta-llama/Llama-2-7b-chat-hf',
# )

In [7]:
import os
import requests

s = requests.Session()

api_base = os.getenv("OPENAI_API_BASE")
token = os.getenv("OPENAI_API_KEY")
url = f"{api_base}/chat/completions"

In [8]:
if USE_SAME_FEW_SHOTS==True:
    data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx").drop(columns=['Unnamed: 0'])
    data_sel_4_few_shot_examples = data_sel_4_few_shot_examples[
        (data_sel_4_few_shot_examples.text_id==5614) |
        (data_sel_4_few_shot_examples.text_id==2089) |
        (data_sel_4_few_shot_examples.text_id==5665) |
        (data_sel_4_few_shot_examples.text_id==370)  |
        (data_sel_4_few_shot_examples.text_id==1422) |
        (data_sel_4_few_shot_examples.text_id==4329) |
        (data_sel_4_few_shot_examples.text_id==4929)
    ]
else:
    data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx").drop(columns=['Unnamed: 0']).sample(5)

data_sel_4_few_shot_examples

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement
250,370,10.1787/9789264272392-3-en,"For example, the poorest 20% of farm household...",2,0.0,9.0,1.000000
986,1422,10.1787/9789264283299-en,"Nevertheless, the average number of physician ...",3,1.0,8.0,0.777778
1456,2089,10.18356/ee52a573-en,"This refers to the manufacture, use and dispos...",12,3.0,29.0,0.812500
3026,4329,10.18356/30e5b8d3-en,With the introduction of FSC standards in the ...,15,0.0,9.0,1.000000
3444,4929,10.18356/5e8977eb-en,"Hence, climate-economy models without explicit...",13,0.0,3.0,1.000000
3936,5614,10.2139/SSRN.1406042,The modern definitions of the inchoate offence...,16,0.0,6.0,1.000000
3974,5665,10.18356/3fe10a08-en,"As a result, both the trade and current accoun...",8,0.0,5.0,1.000000


In [9]:
few_shots = ""

for _, row in data_sel_4_few_shot_examples[['text', 'sdg']].iterrows():
    input_text = "Input text: \""+row['text']+"\"\n"
    output_text = "Output: \"---SDG "+str(row['sdg'])+'---\"\n\n'
    
    few_shots += input_text
    few_shots += output_text
    
print(few_shots)

Input text: "For example, the poorest 20% of farm households in the Philippines tend to be small-scale producers and landless workers whose rice production does not satisfy their own needs and who therefore purchase 77% of the rice that they consume on average (Chapter 4). Regional levels of stunting among children under five years of age, for example, remain worryingly high (Figure 3). While economic growth has enabled significant declines in populations suffering from chronic poverty, for some households, it may have only altered the nature of food insecurity from permanent to transitory, rather than eliminate it entirely."
Output: "---SDG 2---"

Input text: "Nevertheless, the average number of physician contacts per person is not greater than the EU average. For example, home visits to patients by GPs are a regular practice and patients do not usually have to wait long to get access to GPs, although waiting times for more specialised services (e.g. mental health specialists, ophthal

In [10]:
data = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_1020_texts_TEST.xlsx")
data = data.drop(columns=['Unnamed: 0']) #, 'goal_label', 'motivation_for_the_answer', 'goal_label_int', 'sdg_probability'])
data

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0
1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0
1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0
1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0


In [11]:
template_part1 = """[INST]
<<SYS>>
In the following lines, a text enclosed in triple quotes will be given to you.

The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "---SDG 0---".

The following are 7 examples of expected classification given an input text:

"""+few_shots+"""

The text to be processed is:
<</SYS>>

'''
"""

template_part2 = """
'''

<<SYS>>
Answer with label only usign the format:---LABEL---.
<</SYS>>
[/INST]
"""

In [12]:
#data.iloc[181:183]

In [13]:
# data_sel = data #.sample(2)
data_sel = data #.iloc[181:] #.sample(2)

In [16]:
# body = {
#   "model": "meta-llama/Llama-2-7b-chat-hf",
#   "messages": [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "Say 'test'."}],
#   "temperature": 0.0
# }

# with s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body) as resp:
#     print(resp.json())

In [17]:
import time

cnt = 0
# cnt = 181

results = []
for index, row in data_sel.iterrows():
    
    text_id = row['text_id']
    text = row["text"]
    sdg_true = row["sdg"]
    
    query = template_part1 + text + template_part2
    
    body = {
      "model": "meta-llama/Llama-2-13b-chat-hf",
      "messages": [
          {"role": "system", "content": "You are a helpful assistant."}, 
          {"role": "user", "content": query}],
      "temperature": 0.0
    }
    
        
    try:
        result = s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body).json()
        print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'])
    except Exception as e:
        print("#############################################################################")
        print(e)
        print("#############################################################################")
        
    
    result_extended = (cnt, text_id, sdg_true, result)
    
    with open(output_datapath+f'sdg_classification_Llama2_13B_{cnt}.pkl', 'wb') as f:
        pickle.dump(result_extended, f)
    
    results.append(result_extended)
    cnt+=1
    time.sleep(0.5)

0 6668 0   Sure, I'd be happy to help! Here's the input text you provided:

"L train resumes service after derailing from track in Canarsie, Brooklyn  WABC-TVService restored after subway train derails in Brooklyn: officials  PIX11 New York NewsNo injuries in Brooklyn subway derailment  New York Daily NewsView Full Coverage on Google News"

Based on the given input, I would classify it as follows:

---SDG 11---
1 6099 0   Sure, I'd be happy to help! Here's the input text you provided:

"Brooklyn Decker’s SI Swimsuit Cover Photo Shoot in the Maldives Is Beyond Beautiful  Yahoo Life"

Based on the SDGs you provided, I would classify this text as:

---SDG 12---

Here's why: The text is discussing a photo shoot in the Maldives, which is a tourist destination known for its beautiful beaches and crystal-clear waters. The text also mentions "beauty" and "photography," which are both related to the consumption of luxury goods and services. Therefore, this text is most closely related to SDG 12

23 3912 0   Sure! Here's the text with the labels you requested:

---SDG 8---

---SDG 12---

---SDG 13---

---SDG 15---

---SDG 16---

---SDG 2---

I hope this helps! Let me know if you have any other questions.
24 1565 0   Sure, I'd be happy to help! Here's the input text you provided, with the text enclosed in triple quotes:

"More often than not, the joy of tracking down the perfect gift for someone can be swiftly squashed by the ensuing stress of working against a deadline. This is why it's always a good idea to keep an eye on sales events throughout the year: You never know when you might stumble across a beautiful set of table linens for your friend who loves to entertain or a quirky set of notecards for your stationery-obsessed sister.

Amazon Prime Day is one such sales event. This year, it's slated for July 11 and July 12, with discounts on all things travel, home, and fashion—but if you're looking to start your shopping early, there's already plenty of Prime Day gift ideas ma

43 4505 0   Sure, I'd be happy to help! Here's the label for the input text you provided:

---SDG 16---
44 1776 0   Sure, I'd be happy to help! Here's the label for the text you provided:

---SDG 12---
45 1529 0   Sure, I'd be happy to help! Here's the label for the input text you provided:

---SDG 8---
46 1838 0   Sure! Here's the text you provided, with the appropriate SDG label for each paragraph:

---SDG 8---

Jurgen Klopp is currently aiming to revamp his entire midfield.

---SDG 13---

Liverpool target Ryan Gravenberch has admitted that he is keen to stay at Bayern Munich, dealing a blow to the Reds' transfer hopes.

---SDG 16---

Gravenberch, 21, has been at the top of the Anfield outfit's midfield shortlist for the past few months. He was thought to be keen to depart Bayern as he racked up just 937 minutes of action last campaign.

---SDG 2---

However, the Ajax youth product has recently stated that he is aiming to remain at Bayern next season. He told Algemeen Dagblad:

"I'm 

57 193 0   Sure, I'd be happy to help! Here's the input text you provided:

"Final Ariane 5 launch scheduled for July 4 after fixes to booster separation system – Spaceflight Now Spaceflight NowAriane 6 | Mobile gantry removal test timelapse European Space Agency, ESAAriane 6 progress toward inaugural flight: ArianeGroup, Les Mureaux, France European Space AgencyView Full Coverage on Google News"

Based on the content of the text, I would classify it as:

---SDG 9---
58 3497 0   Sure, I'd be happy to help! Here's the label for the text you provided:

---SDG 3---
59 3751 0   Sure, I'd be happy to help! Here's the label for the input text you provided:

---SDG 3---
60 6573 1   Sure, I'd be happy to help! Here's the label for the text you provided:

---SDG 1---
61 357 1   Sure, I'd be happy to help! Here's the label for the input text you provided:

---SDG 1---
62 175 1   Sure, I'd be happy to help! Here's the label for the input text you provided:

---SDG 1---
63 1446 1   Sure, I'd be ha

80 3635 1   Sure, I'd be happy to help! Here's the input text you provided, along with my classification of it according to the Sustainable Development Goals (SDGs):

Input text:

"The definition of national multidimensional child poverty indicators is therefore necessary as a tool to monitor progress towards this SDG target. The headcount is as high as 82 per cent in rural areas, while it is 53 per cent in urban settings. Nationwide 12 per cent of children are not deprived in any dimension. However this is true for only 3 per cent of children in rural areas, while 18 per cent of children in urban areas do not suffer any deprivation."

Output:

---SDG 1---
81 2391 1   Sure, I'd be happy to help! Here's the label for the text you provided:

---SDG 1---
82 841 1   Sure, I'd be happy to help! Here's the label for the text you provided:

---SDG 1---
83 1373 1   Sure, I'd be happy to help! Here's the label for the input text you provided:

---SDG 1---
84 5239 1   Sure, I'd be happy to help!

...

875 2212 14   Sure, I'd be happy to help! Here's the label for the text you provided:

---SDG 15---
876 4014 14   Sure, I'd be happy to help! Here's the label for the input text you provided:

---SDG 14---
877 1200 14   Sure, I'd be happy to help! Here's the label for the text you provided:

---SDG 14---
878 2525 14   Sure, I'd be happy to help! Here's the label for the input text you provided:

---SDG 15---
879 864 14   Sure, I'd be happy to help! Here's the input text you provided:

"There are 161 species (including two subpopulations) listed as threatened (Table 10.1). The great majority of these, 126 species, are listed as Vulnerable (VU), with 27 considered Endangered (EN) and eight species listed as Critically Endangered (CE, a ray, three sharks, two fish and two marine turtle species). The full list of threatened marine species in the WIO is presented in Appendix 10.1."

Based on the content of the text, I would classify it as:

---SDG 15---
880 3804 14   Sure, I'd be happy to h

902 4075 15   Sure, I'd be happy to help! Here's the label for the text you provided:

---SDG 15---
903 3920 15   Sure, I'd be happy to help! Here's the input text you provided, along with my classification of each paragraph according to the Sustainable Development Goals (SDGs):

Input text:

"Australia’s National Strategy for Ecologically Sustainable Development (1992) contains strategic approaches, objectives and actions for various sectors as well as cross-sectoral issues including biodiversity'. One of the core objectives of the strategy is to protect biological diversity and maintain essential ecological processes and life-support systems21 (Australian Government, 1992). For instance, India’s National Action Plan on Climate Change (Gol, 2008) contains eight national missions, two of which specifically relate to biodiversity."

Output:

---SDG 15---

The input text discusses the importance of protecting biodiversity and maintaining ecological processes, which is directly related to

920 1767 15   Sure, I'd be happy to help! Here's the label for the input text you provided:

---SDG 11---
921 1957 15   Sure, I'd be happy to help! Here's the input text you provided, along with my classification of it according to the Sustainable Development Goals (SDGs):

Input text:

"They include altered ecosystems and habitats, detrimental impacts on agriculture, potentially leading to food shortages, and more and longer lasting weather extremes and natural disasters, along with numerous risks to society. Therefore, the regional estimates do not add up to the total. Data collected over two decades show that the growth in global emissions has accelerated, rising 10 per cent from 1990 to 2000 and 38 per cent from 2000 to 2012, driven mostly by growth in the developing regions."

Output:

---SDG 13---
922 980 15   Sure, I'd be happy to help! Here's the label for the input text you provided:

---SDG 15---
923 1299 15   Sure, I'd be happy to help! Here's the input text you provided, al

945 3113 15   Sure, I'd be happy to help! Here's the input text you provided, along with my classification of it according to the Sustainable Development Goals (SDGs):

Input text:

"It compares these financial needs with expected biodiversity expenditures over a medium- to long-term planning horizon. National biodiversity targets are typically articulated in NBSAPs and other key national strategies such as NDPs, sectoral development plans and climate change plans. The plan proposes steps to implement a balanced mix of prioritised finance solutions to sustainably manage biodiversity finance and achieve national biodiversity targets. Based on the findings from the Biodiversity Expenditure Review and the Financial Needs Assessment, the amount that is currently financed is 20% of the total annual requirement."

Output:

---SDG 15---
946 4121 15   Sure, I'd be happy to help! Here's the input text you provided, along with my classification of it according to the Sustainable Development Goal

997 322 16   Sure, I'd be happy to help! Here's the label for the text you provided:

---SDG 16---
998 6351 16   Sure, I'd be happy to help! Here's the label for the text you provided:

---SDG 16---
999 4375 16   Sure, I'd be happy to help! Here's the label for the text you provided:

---SDG 16---
1000 2522 16   Sure, I'd be happy to help! Here's the label for the text you provided:

---SDG 5---
1001 2889 16   Sure, I'd be happy to help! Here's the label for the input text you provided:

---SDG 16---
1002 3927 16   Sure, I'd be happy to help! Here's the label for the text you provided:

---SDG 16---
1003 1248 16   Sure, I'd be happy to help! Here's the label for the text you provided:

---SDG 16---
1004 57 16   Sure, I'd be happy to help! Here's the label for the text you provided:

---SDG 16---
1005 339 16   Sure, I'd be happy to help! Here's the label for the text you provided:

---SDG 16---
1006 4967 16   Sure, I'd be happy to help! Here's the label for the input text you provided:


In [18]:
files = glob.glob(output_datapath + "*.pkl")
# files

In [19]:
# df_list = (pd.DataFrame(pd.read_pickle(file), columns=['cnt', 'id', 'true_label', 'predicted_label']) for file in files)
# df_list = (pd.DataFrame(pd.read_pickle(file)) for file in files)

tuple_list = []
for file in files:
    res = pd.read_pickle(file)
    try:
        if len(res[3]['choices'])>1:
            raise Exception('ERROR: Multilabel detected!')
            break
        sdg_pred = res[3]['choices'][0]['message']['content']
        res4df = (res[0], res[1], res[2], sdg_pred)
        print(res4df)
        tuple_list.append(res4df)
    except: 
        print("#################################################################")
        print(res)
        print("#################################################################")
        

(865, 2639, 14, '  Sure, I\'d be happy to help! Here\'s the input text you provided, along with my classification of it according to the Sustainable Development Goals (SDGs):\n\nInput text:\n\n"The OECD and the FAO will consider them as part of their ongoing work programmes. The Round Table urged caution in attempting to build broader aspects of sustainability (like economic and social sustainability) into an internationally applicable definition applying to fisheries and aquaculture. While these aspects are important at the operational level - particularly in developing countries where adjustments to fisheries management practices will fail if the social and economic impacts are not taken into consideration - they should not be built into an overarching definition or criteria. In the meantime, it was agreed that certification for aspects where there are no agreed definitions, standards, or assessment methodologies are liable to cause confusion."\n\nOutput:\n\n---SDG 12---')
(531, 6741

In [20]:
len(tuple_list)

1019

In [21]:
results_processed = pd.DataFrame(tuple_list, columns=['cnt', 'id', 'true_label', 'predicted_label'])

In [22]:
results_processed

,cnt,id,true_label,predicted_label
0,865,2639,14,"Sure, I'd be happy to help! Here's the input..."
1,531,6741,8,"Sure, I'd be happy to help! Here's the input..."
2,256,1972,4,"Sure, I'd be happy to help! Here's the input..."
3,816,4186,13,"Sure, I'd be happy to help! Here's the label..."
4,444,2876,7,"Sure, I'd be happy to help! Here's the label..."
...,...,...,...,...
1014,667,4353,11,"Sure, I'd be happy to help! Here's the label..."
1015,487,433,8,"Sure, I'd be happy to help! Here's the input..."
1016,34,6228,0,Sure! Here's the text with the labels you re...
1017,907,3261,15,"Sure, I'd be happy to help! Here's the label..."


In [23]:
results_processed_merged = data.merge(results_processed, right_on='id', left_on='text_id')
results_processed_merged

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,0,6668,0,"Sure, I'd be happy to help! Here's the input..."
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,1,6099,0,"Sure, I'd be happy to help! Here's the input..."
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,2,6471,0,"Sure, I'd be happy to help! Here's the label..."
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,3,1786,0,"Sure, I'd be happy to help! Here's the input..."
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN,4,2143,0,"Sure, I'd be happy to help! Here's the label..."
...,...,...,...,...,...,...,...,...,...,...,...
1014,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,1015,4695,16,"Sure, I'd be happy to help! Here's the label..."
1015,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,1016,6628,16,"Sure, I'd be happy to help! Here's the label..."
1016,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,1017,4563,16,"Sure, I'd be happy to help! Here's the label..."
1017,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,1018,6710,16,"Sure, I'd be happy to help! Here's the label..."


In [24]:
results_processed_merged['predicted_label_clean'] = results_processed_merged['predicted_label'].str.extract(r'SDG (\d+)')
results_processed_merged

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label,predicted_label_clean
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,0,6668,0,"Sure, I'd be happy to help! Here's the input...",11
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,1,6099,0,"Sure, I'd be happy to help! Here's the input...",12
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,2,6471,0,"Sure, I'd be happy to help! Here's the label...",16
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,3,1786,0,"Sure, I'd be happy to help! Here's the input...",4
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN,4,2143,0,"Sure, I'd be happy to help! Here's the label...",8
...,...,...,...,...,...,...,...,...,...,...,...,...
1014,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,1015,4695,16,"Sure, I'd be happy to help! Here's the label...",16
1015,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,1016,6628,16,"Sure, I'd be happy to help! Here's the label...",16
1016,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,1017,4563,16,"Sure, I'd be happy to help! Here's the label...",16
1017,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,1018,6710,16,"Sure, I'd be happy to help! Here's the label...",16


In [25]:
results_processed_merged.groupby("predicted_label_clean").count()

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label
predicted_label_clean,,,,,,,,,,,
1,66,66,66,66,66,66,66,66,66,66,66
10,24,24,24,24,24,24,24,24,24,24,24
11,49,47,49,49,47,47,47,49,49,49,49
12,93,83,93,93,83,83,83,93,93,93,93
13,75,71,75,75,71,71,71,75,75,75,75
14,39,39,39,39,39,39,39,39,39,39,39
15,65,63,65,65,63,63,63,65,65,65,65
16,86,75,86,86,75,75,75,86,86,86,86
17,15,15,15,15,15,15,15,15,15,15,15


In [26]:
results_processed_merged.to_excel(output_datapath+'report_sdg_classification_Llama2_13B.xlsx')

In [27]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1019
1019
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        59
           1       0.70      0.77      0.73        60
           2       0.80      0.68      0.74        60
           3       0.71      0.93      0.81        60
           4       0.72      0.97      0.83        60
           5       0.93      0.72      0.81        60
           6       0.98      0.83      0.90        60
           7       0.93      0.68      0.79        60
           8       0.36      0.72      0.48        60
           9       0.76      0.48      0.59        60
          10       0.92      0.37      0.52        60
          11       0.84      0.68      0.75        60
          12       0.57      0.88      0.69        60
          13       0.64      0.80      0.71        60
          14       0.95      0.62      0.75        60
          15       0.78      0.85      0.82        60
          16       0.63      0.90      0.74        60
          17     

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.